In [1]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np


In [2]:
class SimpleFullyConnectedNN(nn.Module):
    def __init__(self, input_size=784, hidden_size=256, num_classes=10):
        super(SimpleFullyConnectedNN, self).__init__()
        # 初始化权重和偏置
        self.W1 = nn.Parameter(torch.randn(input_size, hidden_size) * 0.01)
        self.b1 = nn.Parameter(torch.zeros(hidden_size))
        self.W2 = nn.Parameter(torch.randn(hidden_size, num_classes) * 0.01)
        self.b2 = nn.Parameter(torch.zeros(num_classes))
    
    def forward(self, x):
        # 前向传播
        x = torch.matmul(x, self.W1) + self.b1
        x = torch.relu(x)  # 激活函数
        x = torch.matmul(x, self.W2) + self.b2
        return x


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 26421880/26421880 [00:25<00:00, 1036370.99it/s]


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 124735.64it/s]


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:03<00:00, 1191169.00it/s]


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<00:00, 3525269.71it/s]


Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



In [4]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        for images, labels in train_loader:
            images = images.view(images.shape[0], -1)  # 展平图像
            
            # 前向传播
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [5]:
def accuracy(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(images.shape[0], -1)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')


In [6]:
model = SimpleFullyConnectedNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)
accuracy(model, test_loader)


Epoch [1/10], Loss: 0.2449
Epoch [2/10], Loss: 0.4674
Epoch [3/10], Loss: 0.5073
Epoch [4/10], Loss: 0.1099
Epoch [5/10], Loss: 0.2969
Epoch [6/10], Loss: 0.3947
Epoch [7/10], Loss: 0.2171
Epoch [8/10], Loss: 0.1077
Epoch [9/10], Loss: 0.4774
Epoch [10/10], Loss: 0.1532
Accuracy of the network on the 10000 test images: 88.28 %
